In [ ]:
import geopandas as gpd

df = gpd.read_file('./data/gsi_nummap/523960/DKG-SHP-523960-RailTrCL-20190125-0001.shp', encoding="Shift_JIS")
df = df.explode(index_parts=False)
df.to_file('_hakonetozan_tmp.geojson', index=False, driver='GeoJSON')
# edit tmp_geoj on qgis

In [ ]:
import geopandas as gpd
import networkx as nx

df = gpd.read_file('_hakonetozan_tmp.geojson')
df['_start_node'] = df.geometry.apply(lambda g: str(g.coords[:][0]))
df['_end_node'] = df.geometry.apply(lambda g: str(g.coords[:][-1]))
df.to_file('_hakonetozan_tmp.geojson', index=False, driver='GeoJSON')
g = nx.from_pandas_edgelist(df, '_start_node', '_end_node', ['rID', '_start_node', '_end_node'])
edge_by_rid = {rid: edge for edge, rid in nx.get_edge_attributes(g, 'rID').items()}

def track_links(start_rid, end_rid):
    def get_length(path):
        rids = [g[s][e]['rID'] for s, e in zip(path[:-1], path[1:])]
        return df[df.rID.isin(rids)].geometry.to_crs('EPSG:2451').length.sum()

    start_node_a, start_node_b = edge_by_rid[start_rid]
    end_node_a, end_node_b = edge_by_rid[end_rid]
    # TODO: 複数経路ある場合は、最短経路≠end_ridのedgeにならない場合あり
    # ex. '50067-12684-s-1593', '50061-12687-s-2679_n1' # 上大平台信号場→宮ノ下
    pathes = [
        nx.shortest_path(g, source=start_node_a, target=end_node_a),
        nx.shortest_path(g, source=start_node_a, target=end_node_b),
        nx.shortest_path(g, source=start_node_b, target=end_node_a),
        nx.shortest_path(g, source=start_node_b, target=end_node_b),
    ]
    path_nodes = sorted([(get_length(path), path, i) for i, path in enumerate(pathes)])[-1][1]

    f = lambda snode, enode: 'forward' if g[snode][enode]['_start_node'] == snode else 'backward'
    return [(g[s][e]['rID'], f(s, e)) for s, e in zip(path_nodes[:-1], path_nodes[1:])]

In [ ]:
from shapely.geometry import LineString
from shapely.ops import linemerge
import utils

# gdalbuildvrt _hakone.vrt ./data/dem05s/5239[67][01].tif ./data/dem05s/5238[67]7.tif
get_alt = utils.get_get_alt('./_hakone.vrt')

def get_geom_3d(links):
    geoms = []
    for rid, toward in links:
        row = df[df.rID == rid].iloc[0]
        coords = row.geometry.coords[:]
        coords = coords[::-1] if toward == 'backward' else coords
        if row.railState == 'トンネル':
            a = get_alt(*coords[0])
            geom = LineString([(c[0], c[1], a) for c in coords])
        else:
            geom = LineString([(c[0], c[1], get_alt(*c)) for c in coords])
        geoms.append(geom)
    return linemerge(geoms)

In [ ]:
import folium
import matplotlib.pyplot as plt
import matplotlib.colors as colors

def plot_links_on_map(df, rids):
    def get_line(df, rid, color):
        get_coords = lambda g: [c[::-1] for c in g.coords[:]]
        return folium.vector_layers.PolyLine(
            locations=get_coords(df[df.rID == rid].iloc[0].geometry), 
            tooltip=rid, color=color, weight=4)

    center = df[df.rID == rids[0]].iloc[0].geometry.coords[:][0][::-1]
    cmap = plt.get_cmap('tab10')
    map = folium.Map(location=center, zoom_start=16, max_zoom=19)
    for i, rid in enumerate(rids):
        get_line(df, rid, colors.to_hex(cmap(i % 10))).add_to(map)
    return map

In [ ]:
# links = track_links('50058-12687-s-3051', '50055-12687-s-4335_n1') # 彫刻の森→強羅
# plot_links_on_map(df, [rid for rid, _ in links])

links_list = [
    track_links('50076-12684-s-798_n1', '50073-12684-s-685'), # 箱根湯本→塔ノ沢
    track_links('50073-12684-s-685', '50070-12681-s-1068_n2'), # 塔ノ沢→出山信号場
    track_links('50070-12681-s-1068_n2', '50064-12684-s-2560_n1'), # 出山信号場→大平台
    track_links('50064-12684-s-2560_n1', '50067-12684-s-1593'), # 大平台→上大平台信号場
    track_links('50067-12684-s-1593', '50061-12687-s-2679_n1'), # 上大平台信号場→宮ノ下
    track_links('50061-12687-s-2679_n1', '50058-12684-s-1533_n1'), # 宮ノ下→小涌谷
    track_links('50058-12684-s-1533_n1', '50058-12687-s-3051'), # 小涌谷→彫刻の森
    track_links('50058-12687-s-3051', '50055-12687-s-4335_n1') # 彫刻の森→強羅
]
geoms = [get_geom_3d(links) for links in links_list]
gpd.GeoSeries(geoms).to_file('./data/_train_3d_down.geojson', driver='GeoJSON')

In [ ]:
# links = track_links('50055-12687-s-4336_n1', '50058-12687-s-3050') # 強羅→彫刻の森
# plot_links_on_map(df, [rid for rid, _ in links])

links_list = [
    track_links('50055-12687-s-4336_n1', '50058-12687-s-3050'), # 強羅→彫刻の森
    track_links('50058-12687-s-3050', '50058-12684-s-1534'), # 彫刻の森→小涌谷
    track_links('50058-12684-s-1534', '50061-12687-s-2681'), # 小涌谷→宮ノ下
    track_links('50061-12687-s-2681', '50067-12684-s-1592'), # 宮ノ下→上大平台信号場
    track_links('50067-12684-s-1592', '50064-12684-s-2561'), # 上大平台信号場→大平台
    track_links('50064-12684-s-2561', '50070-12681-s-1068_n2'), # 大平台→出山信号場
    track_links('50070-12681-s-1068_n2', '50073-12684-s-685'), # 出山信号場→塔ノ沢
    track_links('50073-12684-s-685', '50076-12684-s-797_n1') # 塔ノ沢→箱根湯本
]
geoms = [get_geom_3d(links) for links in links_list]
gpd.GeoSeries(geoms).to_file('./data/_train_3d_up.geojson', driver='GeoJSON')

In [ ]:
# cablecar
# links = track_links('50052-12687-s-3035', '50055-12687-s-4649_n1')
# plot_links_on_map(df, [rid for rid, _ in links])

a_links = track_links('50055-12687-s-4649_n1', '50052-12687-s-3035') # 強羅→早雲山
geoms = [get_geom_3d(links) for links in [a_links]]
gpd.GeoSeries(geoms).to_file('_cablecar_3d_down.geojson', driver='GeoJSON')

a_links = track_links('50052-12687-s-3035', '50055-12687-s-4649_n1') # 早雲山→強羅
geoms = [get_geom_3d(links) for links in [a_links]]
gpd.GeoSeries(geoms).to_file('_cablecar_3d_up.geojson', driver='GeoJSON')

In [ ]:
# ropeway
# links = track_links('50040-12687-s-1584', '50040-12687-s-1583') # 早雲山→大涌谷→早雲山
# plot_links_on_map(df, [rid for rid, _ in links])

a_links = track_links('50052-12687-s-3032', '50052-12687-s-3031') # 早雲山→大涌谷→早雲山
geoms = [get_geom_3d(links) for links in [a_links]]
gpd.GeoSeries(geoms).to_file('_ropeway_3d_1.geojson', driver='GeoJSON')

a_links = track_links('50046-12687-s-2383', '50046-12687-s-2384') # 大涌谷→姥子→大涌谷
geoms = [get_geom_3d(links) for links in [a_links]]
gpd.GeoSeries(geoms).to_file('_ropeway_3d_2.geojson', driver='GeoJSON')

a_links = track_links('50040-12687-s-1584', '50040-12687-s-1583') # 姥子→桃源台→姥子
geoms = [get_geom_3d(links) for links in [a_links]]
gpd.GeoSeries(geoms).to_file('_ropeway_3d_3.geojson', driver='GeoJSON')

In [ ]:
from shapely.geometry import LineString
from shapely.ops import substring
from shapely.ops import transform
from pyproj import Transformer

to_meter = Transformer.from_crs('EPSG:4612', 'EPSG:2451', always_xy=True).transform
to_latlon = Transformer.from_crs('EPSG:2451', 'EPSG:4612', always_xy=True).transform

def get_cartgraphic_degrees(geoms, offset, height, timetable):
    def get_distances(geom):
        lengths = [LineString(geom.coords[i:i+2]).length for i in range(len(geom.coords) - 1)]
        dists = [0] + [sum(lengths[:i]) for i in range(2, len(lengths) + 1)]
        adists = [lengths[0] / 2] + [sum(lengths[:i]) for i in range(2, len(lengths))] + [sum(lengths) - lengths[-1] / 2]
        minmax = lambda d: max(min(d, dists[-1]), 0)
        adists = [minmax(d + offset) for d in adists]
        return dists, adists

    def get_section(geom, height, start, end):
        geom = transform(to_meter, geom)
        dists, adists = get_distances(geom)
        fsec = lambda d: (end - start) * (d / dists[-1]) + start
        fpos = lambda d: substring(geom, start_dist=d, end_dist=d).coords[0]
        secs = [fsec(d) for d in dists]
        poss = transform(to_latlon, LineString([fpos(d) for d in adists])).coords
        return [(sec, x, y, z + height) for sec, (x, y, z) in zip(secs, poss)]

    sections = [get_section(geom, height, start, end) for geom, (start, end) in zip(geoms, timetable)]
    return [a for section in sections for pos in section for a in pos]

In [89]:
def get_header(name):
    return {
        'id': 'document',
        'name': name,
        'version': '1.0'
    }

def get_body(aid, name, epoch, flat_poss_list, box=None, ellipsoid=None):
    body = {
        'id': aid,
        'name': name,
        'position': {
            'epoch': epoch.strftime('%Y-%m-%dT%H:%M:%S+09'),
            'cartographicDegrees': flat_poss_list
        },
        'orientation': {
            'velocityReference': "%s#position" % aid
        },
        'forwardExtrapolationType': 'None',
        'backwardExtrapolationType': 'None',
    }
    if box:
        body['box'] = box
    if ellipsoid:
        body['ellipsoid'] = ellipsoid
    return body

def get_czml_train_body(aid, name, description, epoch, cart_degs):
    return {
        'id': aid,
        'name': name,
        'description': description,
        'position': {
            'epoch': epoch.strftime('%Y-%m-%dT%H:%M:%S+09'),
            'cartographicDegrees': cart_degs
        },
        'orientation': {
            'velocityReference': f"{aid}#position"
        },
        'forwardExtrapolationType': 'None',
        'backwardExtrapolationType': 'None',
        'model': {
            'gltf': 'data/vehicles/train.glb'
        }
    }


In [ ]:
# train shape
# https://www.hakonenavi.jp/hakone-tozan/type_moha1/
TRAIN_SIZE = [14.66, 2.568, 3.990 - (0.8 + 0.4)]  # length, width, height
TRAIN_BOX = {
    'dimensions': { 'cartesian': TRAIN_SIZE },
    'material': { 'solidColor': { 'color': { 'rgba': [255, 228, 196, 0xff] } } },
    'shadows': 'ENABLED'
}
TRAIN_OFFSET = TRAIN_SIZE[0] / 2 + 0.2
TRAIN_HEIGHT = 40.0222 + TRAIN_SIZE[2] / 2 + 0.8

In [86]:
from datetime import datetime as dt, timedelta as td

STATIONS = ['箱根湯本', '塔ノ沢', '大平台', '宮ノ下', '小涌谷', '彫刻の森', '強羅']
DULATIONS = [0, 4, 11, 11, 5, 3, 3]

def get_description(deperture):
    n = dt.now()
    start = dt.strptime(f'{n.year}-{n.month}-{n.day}T{deperture}', '%Y-%m-%dT%H:%M')
    lines = []
    for i, (s, d) in enumerate(zip(STATIONS, DULATIONS)):
        t = start + td(minutes=d)
        ad = '発' if i < len(STATIONS) - 1 else ''
        ad += '着' if 0 < i else ''
        lines.append(f'{s} {t.hour:02}:{t.minute:02}{ad}')
    return ('<br>').join(lines)

In [87]:
import json
from datetime import datetime as dt, timedelta as td
import geopandas as gpd

# make train down czml

TIMETABLE = [
    [0, 180], # 箱根湯本→塔ノ沢
    [195, 540], # 塔ノ沢→出山信号場
    [555, 900], # 出山信号場→大平台
    [915, 1080], # 大平台→上大平台信号場
    [1095, 1500], # 上大平台信号場→宮ノ下
    [1515, 1800], # 宮ノ下→小涌谷
    [1815, 1980], # 小涌谷→彫刻の森
    [1985, 2160], # 彫刻の森→強羅
]
DEPERTURES = [
    '05:50',
    '06:12', '06:34',
    '07:00', '07:18', '07:36', '07:46',
    '08:00', '08:24', '08:45',
    '09:05', '09:25', '09:37', '09:50',
    '10:11', '10:24', '10:38', '10:49',
    '11:09', '11:22', '11:36', '11:49',
    '12:10', '12:25', '12:38', '12:52',
    '13:12', '13:25', '13:39', '13:51',
    '14:12', '14:24', '14:38', '14:50',
    '15:12', '15:25', '15:39', '15:52',
    '16:08', '16:20', '16:32', '16:44', '16:56',
    '17:08', '17:20', '17:32', '17:44', '17:56',
    '18:08', '18:20', '18:32', '18:44', '18:56',
    '19:08', '19:20', '19:32', '19:44', '19:56',
    '20:08', '20:20', '20:32', '20:44', '20:56',
    '21:08', '21:20', '21:32', '21:44', '21:57',
    '22:10', '22:34', 
    '23:00', '23:26', '23:56'
]

geoms = gpd.read_file('./data/_train_3d_down.geojson').geometry.to_list()
cart_degs_1 = get_cartgraphic_degrees(geoms, TRAIN_OFFSET, TRAIN_HEIGHT, TIMETABLE)
cart_degs_2 = get_cartgraphic_degrees(geoms, -TRAIN_OFFSET, TRAIN_HEIGHT, TIMETABLE)
packets = [get_header('箱根登山電車')]
n = dt.now()
for i, deperture in enumerate(DEPERTURES):
    epoch = dt.strptime(f'{n.year}-{n.month}-{n.day}T{deperture}', '%Y-%m-%dT%H:%M')
    description = get_description(deperture)
    packets += [
        get_czml_train_body(f"{i}_d_1", "箱根登山電車 強羅行き", description, epoch, cart_degs_1),
        get_czml_train_body(f"{i}_d_2", "箱根登山電車 強羅行き", description, epoch, cart_degs_2)
    ]
open('./dst/train_down_czml.json', 'w').write(json.dumps(packets, indent=2))

16031669

In [88]:
import json
from datetime import datetime as dt, timedelta as td
import geopandas as gpd

# make train up czml

TIMETABLE = [
    [0, 120], # 強羅→彫刻の森
    [135, 300], # 彫刻の森→小涌谷
    [315, 600], # 小涌谷→宮ノ下
    [615, 900], # 宮ノ下→上大平台信号場
    [915, 1200], # 上大平台信号場→大平台
    [1215, 1620], # 大平台→出山信号場
    [1635, 1980], # 出山信号場→塔ノ沢
    [1995, 2220], # 塔ノ沢→箱根湯本
]
DEPERTURES = [
    '05:50',
    '06:12', '06:34',
    '07:00', '07:18', '07:36', '07:46',
    '08:00', '08:24', '08:45',
    '09:05', '09:25', '09:37', '09:50',
    '10:11', '10:24', '10:38', '10:49',
    '11:09', '11:22', '11:36', '11:49',
    '12:10', '12:25', '12:38', '12:52',
    '13:12', '13:25', '13:39', '13:51',
    '14:12', '14:24', '14:38', '14:50',
    '15:12', '15:25', '15:39', '15:52',
    '16:08', '16:20', '16:32', '16:44', '16:56',
    '17:08', '17:20', '17:32', '17:44', '17:56',
    '18:08', '18:20', '18:32', '18:44', '18:56',
    '19:08', '19:20', '19:32', '19:44', '19:56',
    '20:08', '20:20', '20:32', '20:44', '20:56',
    '21:08', '21:20', '21:32', '21:44', '21:57',
    '22:10', '22:34', 
    '23:00', '23:26', '23:56'
]

geoms = gpd.read_file('./data/_train_3d_up.geojson').geometry.to_list()
cart_degs_1 = get_cartgraphic_degrees(geoms, TRAIN_OFFSET, TRAIN_HEIGHT, TIMETABLE)
cart_degs_2 = get_cartgraphic_degrees(geoms, -TRAIN_OFFSET, TRAIN_HEIGHT, TIMETABLE)
packets = [get_header('箱根登山電車')]
n = dt.now()
for i, deperture in enumerate(DEPERTURES):
    epoch = dt.strptime(f'{n.year}-{n.month}-{n.day}T{deperture}', '%Y-%m-%dT%H:%M')
    description = get_description(deperture)
    packets += [
        get_czml_train_body(f"{i}_u_1", "箱根登山電車 小田原行き", description, epoch, cart_degs_1),
        get_czml_train_body(f"{i}_u_2", "箱根登山電車 小田原行き", description, epoch, cart_degs_1)
    ]
open('./dst/train_up_czml.json', 'w').write(json.dumps(packets, indent=2))

16123138

In [ ]:
import json
from datetime import datetime
import geopandas as gpd

# make cablecar down czml
box = {
    'dimensions': { 'cartesian': [12, 2.42, 3.58] },
    'material': { 'solidColor': { 'color': { 'rgba': [255, 69, 0, 0xff] } } },
    'shadows': 'ENABLED'
}

df = gpd.read_file('_cablecar_3d_down.geojson')

timetable = [
    [0, 600], # 強羅→早雲山
]
starts = [
    '07:41',
    '08:00', '08:20', '08:45',
    '09:05', '09:21', '09:36', '09:51',
    '10:06', '10:21', '10:36', '10:54',
    '11:17', '11:31', '11:54',
    '12:17', '12:31', '12:54',
    '13:17', '13:31', '13:54',
    '14:17', '14:31', '14:54',
    '15:17', '15:31', '15:54',
    '16:17', '16:31', '16:54',
    '17:17', '17:31', '17:54',
    '18:20', '18:35',
    '19:07'
]

cartgraphic_degrees_1 = get_cartgraphic_degrees(df.geometry.to_list(), 6.2, 40.0222 + 2.79, timetable)
cartgraphic_degrees_2 = get_cartgraphic_degrees(df.geometry.to_list(), -6.2, 40.0222 + 2.79, timetable)

epochs = [datetime.strptime("2022-11-19T%s" % start, '%Y-%m-%dT%H:%M') for start in starts]
packets = [get_header('ケーブルカー1号車')]
for i, epoch in enumerate(epochs):
    packets += [
        get_body(f"{i * 2 + 1}M_1", "江ノ電 鎌倉行き", epoch, cartgraphic_degrees_1, box=box),
        get_body(f"{i * 2 + 1}M_2", "江ノ電 鎌倉行き", epoch, cartgraphic_degrees_2, box=box),
    ]
open('cablecar_down_czml.json', 'w').write(json.dumps(packets, indent=2))

In [ ]:
import json
from datetime import datetime
import geopandas as gpd

# make cablecar up czml
box = {
    'dimensions': { 'cartesian': [12, 2.42, 3.58] },
    'material': { 'solidColor': { 'color': { 'rgba': [30, 144, 255, 0xff] } } },
    'shadows': 'ENABLED'
}

df = gpd.read_file('_cablecar_3d_up.geojson')

timetable = [
    [0, 600], # 早雲山→強羅
]
starts = [
    '07:41',
    '08:00', '08:20', '08:45',
    '09:05', '09:21', '09:36', '09:51',
    '10:06', '10:21', '10:36', '10:54',
    '11:17', '11:31', '11:54',
    '12:17', '12:31', '12:54',
    '13:17', '13:31', '13:54',
    '14:17', '14:31', '14:54',
    '15:17', '15:31', '15:54',
    '16:17', '16:31', '16:54',
    '17:17', '17:31', '17:54',
    '18:20', '18:35',
    '19:07'
]

cartgraphic_degrees_1 = get_cartgraphic_degrees(df.geometry.to_list(), 6.2, 40.0222 + 2.79, timetable)
cartgraphic_degrees_2 = get_cartgraphic_degrees(df.geometry.to_list(), -6.2, 40.0222 + 2.79, timetable)

epochs = [datetime.strptime("2022-11-19T%s" % start, '%Y-%m-%dT%H:%M') for start in starts]
packets = [get_header('ケーブルカー2号車')]
for i, epoch in enumerate(epochs):
    packets += [
        get_body(f"{i * 2 + 2}M_1", "江ノ電 鎌倉行き", epoch, cartgraphic_degrees_1, box=box),
        get_body(f"{i * 2 + 2}M_2", "江ノ電 鎌倉行き", epoch, cartgraphic_degrees_2, box=box),
    ]
open('cablecar_up_czml.json', 'w').write(json.dumps(packets, indent=2))

In [93]:
import json
from datetime import datetime
import geopandas as gpd

# make ropeway down czml

def get_czml_ropeway_body(aid, name, description, epoch, cart_degs):
    return {
        'id': aid,
        'name': name,
        'description': description,
        'position': {
            'epoch': epoch.strftime('%Y-%m-%dT%H:%M:%S+09'),
            'cartographicDegrees': cart_degs
        },
        'orientation': {
            'velocityReference': f"{aid}#position"
        },
        'forwardExtrapolationType': 'None',
        'backwardExtrapolationType': 'None',
        'model': {
            'gltf': 'data/vehicles/ropeway.glb'
        }
    }

timetable = [[0, 1800]]  # 早雲山→大涌谷→早雲山
starts = [f'{h:02}:{m:02}' for h in range(8, 18) for m in range(0, 60, 2)]
epochs = [datetime.strptime("2022-11-19T%s" % start, '%Y-%m-%dT%H:%M') for start in starts]

df = gpd.read_file('./data/_ropeway_3d_1.geojson')
cartgraphic_degrees = get_cartgraphic_degrees(df.geometry.to_list(), 0, 40.0222 + 12.5, timetable)
packets = [get_header('ロープウェイ')]
for i, epoch in enumerate(epochs):
    packets += [
        get_czml_ropeway_body(f"{i}_1", "箱根ロープウェイ 早雲山駅〜大涌谷駅", '', epoch, cartgraphic_degrees),
    ]
open('./dst/ropeway_1_czml.json', 'w').write(json.dumps(packets, indent=2))

df = gpd.read_file('./data/_ropeway_3d_2.geojson')
cartgraphic_degrees = get_cartgraphic_degrees(df.geometry.to_list(), 0, 40.0222 + 12.5, timetable)
packets = [get_header('ロープウェイ')]
for i, epoch in enumerate(epochs):
    packets += [
        get_czml_ropeway_body(f"{i}_2", "箱根ロープウェイ 大涌谷駅〜姥子駅", '', epoch, cartgraphic_degrees),
    ]
open('./dst/ropeway_2_czml.json', 'w').write(json.dumps(packets, indent=2))

df = gpd.read_file('./data/_ropeway_3d_3.geojson')
cartgraphic_degrees = get_cartgraphic_degrees(df.geometry.to_list(), 0, 40.0222 + 12.5, timetable)
packets = [get_header('ロープウェイ')]
for i, epoch in enumerate(epochs):
    packets += [
        get_czml_ropeway_body(f"{i}_3", "箱根ロープウェイ 姥子駅〜桃源台駅", '', epoch, cartgraphic_degrees),
    ]
open('./dst/ropeway_3_czml.json', 'w').write(json.dumps(packets, indent=2))


430386

In [ ]:
print a